<a href="https://colab.research.google.com/github/BenedettaFrancesconi/Granger_DeepAR_Indonesia/blob/main/Model_X_Knockoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data retrieval and preparation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
Indonesia = pd.read_csv('Indonesia_timeseries_mean.csv')

In [ ]:
## Set the time as an index if needed


Indonesia['time'] = pd.to_datetime(Indonesia['time'])
Indonesia.index = Indonesia['time']
Indonesia.drop(['time','areacella'],axis='columns', inplace=True)
Indonesia.head(5)

,cropFrac,gpp,grassFrac,nbp,npp,pastureFrac,treeFrac,nep,pr,rsds,rsus,tas
time,,,,,,,,,,,,
1850-01-16 12:00:00,0.655965,4.923347e-08,7.878640,-3.374646e-09,2.103969e-08,0.115874,12.365179,-2.800102e-09,0.000127,204.66885,13.699398,299.48627
1850-02-15 00:00:00,0.655965,5.173679e-08,7.948023,1.254644e-09,2.432577e-08,0.115874,12.368155,1.938519e-09,0.000120,206.91608,13.525014,299.32812
1850-03-16 12:00:00,0.655965,5.405059e-08,8.028105,3.827634e-09,2.587070e-08,0.115874,12.369579,4.487882e-09,0.000123,208.66682,13.711241,299.33707
1850-04-16 00:00:00,0.655965,5.238121e-08,8.111268,2.724934e-09,2.393364e-08,0.115874,12.373181,3.417906e-09,0.000116,209.31406,13.727771,299.73420
1850-05-16 12:00:00,0.655965,5.258140e-08,8.176949,1.705504e-09,2.430349e-08,0.115874,12.378870,2.285713e-09,0.000078,211.94884,14.260654,299.64816


In [ ]:
for c in Indonesia.columns:
  print(c)

cropFrac
gpp
grassFrac
nbp
npp
pastureFrac
treeFrac
nep
pr
rsds
rsus
tas


In [ ]:
## check again stationarity

from statsmodels.tsa.stattools import adfuller

for c in Indonesia.columns:
  #columns = []
  result = adfuller(Indonesia[c])
  if result[1]>0.05:
    columns = columns
    print(f'{c}')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    columns.append(c)
    print()
  else:
    pass
print(columns)

cropFrac
ADF Statistic: 4.551206
p-value: 1.000000
Critical Values:

grassFrac
ADF Statistic: -2.162340
p-value: 0.220197
Critical Values:

pastureFrac
ADF Statistic: -0.802126
p-value: 0.818497
Critical Values:

treeFrac
ADF Statistic: 3.658358
p-value: 1.000000
Critical Values:

['treeFrac', 'cropFrac', 'grassFrac', 'pastureFrac', 'treeFrac']


In [ ]:
## check again stationarity taking the first difference

columns = ['treeFrac', 'cropFrac', 'grassFrac', 'pastureFrac']

for c in columns:
  x = Indonesia[c].diff()
  result = adfuller(x[1:])
  if result[1]<0.05:
    print(f'{c}')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    print()
  else:
    pass

In [ ]:
## check again stationarity but detrending the data

columns = ['treeFrac', 'cropFrac', 'grassFrac', 'pastureFrac']

for c in columns:
  x = signal.detrend(Indonesia[c])
  result = adfuller(x)
  if result[1]>0.05:
    print(f'{c}')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    print()
  else:
    pass

In [ ]:
## in many cases both the pvalue> alpha and t_c>critical values, so we can't reject the H_{0}: unit root present and non stationary
## when taking the first difference, the non stationary variables ['treeFrac', 'cropFrac', 'grassFrac', 'pastureFrac'] become
## stationary

In [ ]:
for c in columns:
  plt.plot(Indonesia[c])
  plt.show()

In [ ]:
for c in columns:
  plt.plot(np.log(Indonesia[c]))
  plt.show()

In [ ]:
for c in columns:
  plt.plot(signal.detrend(Indonesia[c]))
  plt.show()

# Model-X Knockoffs

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

install.packages("knockoff")
library(knockoff)

In [ ]:
# %%R

# set.seed(1234)
# n = 1000          # number of observations
# p = 1000          # number of variables
# k = 60            # number of variables with nonzero coefficients
# amplitude = 4.5   # signal amplitude (for noise level = 1)

# # Generate the variables from a multivariate normal distribution
# mu = rep(0,p)
# rho = 0.25
# Sigma = toeplitz(rho^(0:(p-1)))
# X = matrix(rnorm(n*p),n) %*% chol(Sigma)

# # Generate the response from a linear model
# nonzero = sample(p, k)
# beta = amplitude * (1:p %in% nonzero) / sqrt(n)
# y.sample = function(X) X %*% beta + rnorm(n)
# y = y.sample(X)

In [ ]:
%%R
install.packages('doParallel')
library(doParallel)

result = knockoff.filter(X, y)
print(result)

In [ ]:
%%R

library(readr)
Indonesia <- read_csv("Indonesia_timeseries_mean.csv")
#columns_to_drop <- ['gpp','nbp','npp','nep','areacella']
within(Indonesia, rm('time','gpp','nbp','npp','nep','areacella'))

Rows: 1980 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (13): cropFrac, gpp, grassFrac, nbp, npp, pastureFrac, treeFrac, nep, p...
dttm  (1): time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 1,980 × 8
   cropFrac grassFrac pastureFrac treeFrac        pr  rsds  rsus   tas
      <dbl>     <dbl>       <dbl>    <dbl>     <dbl> <dbl> <dbl> <dbl>
 1    0.656      7.88       0.116     12.4 0.000127   205.  13.7  299.
 2    0.656      7.95       0.116     12.4 0.000120   207.  13.5  299.
 3    0.656      8.03       0.116     12.4 0.000123   209.  13.7  299.
 4    0.656      8.11       0.116     12.4 0.000116   209.  13.7  300.
 5    0.656      8.18       0.116     12.4 0.0000781  212.  14.3  300.
 6    0.656      8.22       0.116     12.4 0.0000585  202.  14.0  299.
 7    0.656      8.23       0.116     12

In [ ]:
%%R
p = 8
n = 1980
mu = rep(0,p)
rho = 0.10
Sigma = toeplitz(rho^(0:(p-1)))
X = matrix(rnorm(n*p),n) %*% chol(Sigma)

In [ ]:
%%R

X_k = create.gaussian(X, mu, Sigma)

In [ ]:
%%R

X_k